In [77]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException,ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import random
import re
# Setup ChromeDriver
options = webdriver.ChromeOptions()
# options.add_argument('--disable-blink-features=AutomationControlled')  # Disable automation flags
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_experimental_option(
        "prefs", {
            # block image loading
            "profile.managed_default_content_settings.images": 2,
        }
    )
# options.add_argument('--headless')  # Uncomment for headless mode

service = Service(ChromeDriverManager().install())

# driver = webdriver.Chrome(service=service, options=options)
# wait = WebDriverWait(driver, 10)
# driver.get("https://nhadatvui.vn/ban-nha-rieng-phuong-thuan-phuoc-quan-hai-chau/ban-can-ho-5-tang-goc-2-mat-tien-duong-cao-xuan-duc-ngay-du-an-cv-phan-mem-so-2-da-nang1727338496")


In [78]:
data_list = []
STANDARD_KEYS = ['price','Diện tích', 'Chiều rộng','Chiều dài','Năm xây dựng', 'Giấy tờ pháp lý','Hiện trạng nhà',
                 'Phòng ngủ','Phòng tắm', 'Vị trí', 'Trạng thái sử dụng', 'Số tầng', 'Đường rộng','Hướng','Kết cấu']

In [79]:
def scrape_page():
    data_dict = {}
    
    wait.until(EC.presence_of_all_elements_located((By.XPATH, '//ul[contains(@class,"list-full-info-product")]/li')))
    info = driver.find_elements(By.XPATH, '//*[@id="content-tab-info"]/div/div/ul/li')
    price = driver.find_element(By.CLASS_NAME, 'price').text.strip()
    data_dict['price'] = price
    print(len(info))
    for li in info:
        try:
            # Find the 'span' elements within the current 'li' element
            key = li.find_element(By.XPATH, "./span[1]").text.strip()  
            val = li.find_element(By.XPATH, "./span[2]").text.strip()

            data_dict[key] = val
        except (NoSuchElementException, StaleElementReferenceException) as e:
            print(f"Error processing element: {e}")
            continue
    
    for key in STANDARD_KEYS:
        if key not in data_dict:
            data_dict[key] = None

    data_list.append(data_dict)
    print(data_list)
# scrape_page()

In [81]:
def remove_header():
    try:
        driver.execute_script("""
            var header = document.querySelector('.px-4');
            if (header) {
                header.remove();
            }
        """)
        print("Header element removed.")
    except Exception as e:
        print(f"Failed to remove header element: {e}")
def remove_search_form():
    script = """
    var element = document.querySelector('.px-8');
    if (element) {
        element.remove();
    }
    """
    driver.execute_script(script)
    print("Search form removed.")
# Human-like delay function
def human_like_delay(min_time=1, max_time=3):
    time.sleep(random.uniform(min_time, max_time))

def scroll_to_bottom():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    human_like_delay(1, 3)


In [82]:
def scrape_main_page(url):
    human_like_delay(1, 3)
    driver.get(url)
    scroll_to_bottom()
    human_like_delay(2, 4)

    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.box-item.box-item-hover.border.rounded.bg-white')))

    remove_header()
    remove_search_form()

    cards = driver.find_elements(By.CSS_SELECTOR, '.box-item.box-item-hover.border.rounded.bg-white')
    card_links = []

    for index, card in enumerate(cards):
        try:
            driver.execute_script("arguments[0].scrollIntoView(true);", card)
            human_like_delay(1, 3)

            link_element = card.find_element(By.CSS_SELECTOR, 'a.p-2.pb-3.grid')
            link = link_element.get_attribute('href')
            
            if link:
                card_links.append(link)
                print(f"Collected link for card {index + 1}: {link}")
            else:
                print(f"No href found for card {index + 1}.")
        except NoSuchElementException:
            print(f"Link not found for card {index + 1}.")
        except TimeoutException:
            print(f"Timeout: Link not found for card {index + 1}.")
        except StaleElementReferenceException:
            print(f"Stale reference for card {index + 1}. Re-fetching elements.")
            return []  # To force re-fetching elements
        except Exception as e:
            print(f"An error occurred while collecting link for card {index + 1}: {e}")

    return card_links
def scrape_individual_cards(card_links):
    for index, link in enumerate(card_links):
        try:
            print(f"Scraping card {index + 1} / {len(card_links)}: {link}")
            driver.get(link)
            human_like_delay(2, 4)
            scrape_page()
        except Exception as e:
            print(f"An error occurred while scraping card {index + 1}: {e}")


In [83]:
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)
try:
    for i in range(1,181):
        try:

            url = f'https://nhadatvui.vn/mua-ban-nha-rieng-tp-da-nang?ctIds=14%2C25%2C6&page={i}'
            card_links = scrape_main_page(url)
            scrape_individual_cards(card_links)
        except:
            break
except Exception:
    print("An error occurred while scraping the main page.")
finally:
    print(data_list)
    df = pd.DataFrame(data_list)
    df.to_csv('scraped_data_only_gianha_2.csv',encoding="utf-8", index=False)
    print("Saved successfully")
driver.quit()


Header element removed.
Search form removed.
Collected link for card 1: https://nhadatvui.vn/ban-nha-mat-pho-phuong-binh-thuan-quan-hai-chau/gia-dinh-ban-nha-mat-pho-duong-hoang-dieu-tp-da-nang1710165749
Collected link for card 2: https://nhadatvui.vn/ban-nha-rieng-phuong-hoa-cuong-bac-quan-hai-chau/-2-mat-tien-duong-7-5m-binh-an-hoa-cuong-bac-188m2-ngang-8m-tang-20-phong-nhinh-10-ty1727339718
Collected link for card 3: https://nhadatvui.vn/ban-nha-rieng-phuong-an-hai-bac-quan-son-tra/-mat-tien-duong-dinh-nghe-khu-pho-du-lich-sat-bien-88m2-4-tang-12-x-ty1727338517
Collected link for card 4: https://nhadatvui.vn/ban-nha-rieng-phuong-thuan-phuoc-quan-hai-chau/ban-can-ho-5-tang-goc-2-mat-tien-duong-cao-xuan-duc-ngay-du-an-cv-phan-mem-so-2-da-nang1727338496
Collected link for card 5: https://nhadatvui.vn/ban-nha-rieng-phuong-my-an-quan-ngu-hanh-son/-ban-lo-goc-2mt-khu-pho-tay-an-thuong-duong-ngo-thi-si-dong-tien-50tr-thang-ngang-26m-1727338333
Collected link for card 6: https://nhadatvui.v